In [1]:
# This jupyter-notebook is for my study. Codes was modified based on the following references
# # https://pypi.org/project/keras-transformer/
# https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

# Set up environment

In [2]:
import os

# make only specific GPU to be utilized
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

# select GPU to run on
GPU = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = GPU

# stop GPU
# os.environ["CUDA_VISIBLE_DEVICES"] ="-1"

# set GPU to be deterministic 
os.environ["PYTHONHASHSEED"] = "0" # set hash environment
os.environ["TF_CUDNN_USE_AUTOTUNE"] = "0" # use cuDNN function to retrieve the best algorithm
os.environ["TF_CUDNN_CONVOLUTION_BWD_FILTER_ALGO_DETERMINISTIC"]='1' # use cuDNN deterministic algorithms
os.environ['TF_DETERMINISTIC_OPS'] = '1' # deterministic setting

# set numpy, python, tensorflow random seed
import numpy as np
from numpy.random import seed
import random
random.seed(10)
seed(10)
from tensorflow import set_random_seed
set_random_seed(20)

# GPU memory control
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.40
sess = tf.Session(config=config)

# Set up parameters

In [3]:
# dataset was download from http://www.manythings.org/anki/

In [4]:
# Set up path to the data textfile
data_path = 'data/jpn-eng/jpn.txt'
# data_path = 'data/tha-eng/tha.txt'
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
print ("total line", len(lines))

total line 45449


In [5]:
# Set up parameters for training
batch_size = 64  # Batch size for training.
epochs = 200  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
start_samples1 = 0
end_samples1 = 5000 
start_samples2 = 40000
end_samples2 = 45000 

# Process data

In [6]:
from __future__ import print_function
 
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
import numpy as np
 
# Generate dictionaries
def build_token_dict(token_list):
    token_dict = {
        '<PAD>': 0,
        '<START>': 1,
        '<END>': 2,
    }
    for tokens in token_list:
        for token in tokens:
            if token not in token_dict:
                token_dict[token] = len(token_dict)
    return token_dict

source_input = []
source_tokens = []
target_tokens = []

for line in lines[start_samples1: min(end_samples1, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')       
    source_input.append(input_text)
    source_tokens.append(input_text.split(' '))
    target_tokens.append(target_text.split(' '))

for line in lines[start_samples2: min(end_samples2, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')       
    source_input.append(input_text)
    source_tokens.append(input_text.split(' '))
    target_tokens.append(target_text.split(' '))
    
source_token_dict = build_token_dict(source_tokens)
target_token_dict = build_token_dict(target_tokens)
source_token_dict_inv = {v: k for k, v in source_token_dict.items()}
target_token_dict_inv = {v: k for k, v in target_token_dict.items()}


# Add special tokens
encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
decode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]
output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]

# Padding
source_max_len = max(map(len, encode_tokens))
target_max_len = max(map(len, decode_tokens))

encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
decode_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in decode_tokens]
output_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in output_tokens]

encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens]
decode_input = [list(map(lambda x: target_token_dict[x], tokens)) for tokens in decode_tokens]
decode_output = [list(map(lambda x: [target_token_dict[x]], tokens)) for tokens in output_tokens]

Using TensorFlow backend.


In [7]:
from keras_transformer import get_model
from keras import optimizers

# Build the model
model = get_model(
    token_num=max(len(source_token_dict), len(target_token_dict)),
    embed_dim=32,
    encoder_num=2,
    decoder_num=2,
    head_num=4,
    hidden_dim=128,
    dropout_rate=0.05,
    use_same_embed=False,  # Use different embeddings for different languages
)

# adam_opt = optimizers.Adam(lr=0.0005)

model.compile(
    optimizer='Adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Encoder-Token-Embedding (Embedd [(None, None, 32), ( 278016      Encoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Embedding (TrigPosEmbed (None, None, 32)     0           Encoder-Token-Embedding[0][0]    
__________________________________________________________________________________________________
Encoder-1-MultiHeadSelfAttentio (None, None, 32)     4224        Encoder-Embedding[0][0]          
____________________________________________________________________________________________

In [8]:
import time

# Train the model
start = time.time()
model_history = model.fit([np.array(encode_input), np.array(decode_input)], 
                          np.array(decode_output),
                          epochs=epochs,
                          validation_split=0.2)
end = time.time()
print ("calculation time: ", end - start)

W1222 10:41:41.822609 139836474173248 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1222 10:41:48.488249 139836474173248 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 8000 samples, validate on 2000 samples
Epoch 1/200
8000/8000 [==============================] - 15s 2ms/step - loss: 2.0931 - accuracy: 0.5981 - val_loss: 1.3244 - val_accuracy: 0.6651
Epoch 2/200
8000/8000 [==============================] - 10s 1ms/step - loss: 1.3148 - accuracy: 0.6661 - val_loss: 1.4172 - val_accuracy: 0.6654
Epoch 3/200
8000/8000 [==============================] - 10s 1ms/step - loss: 1.2832 - accuracy: 0.6663 - val_loss: 1.3849 - val_accuracy: 0.6658
Epoch 4/200
8000/8000 [==============================] - 10s 1ms/step - loss: 1.2527 - accuracy: 0.6666 - val_loss: 1.4812 - val_accuracy: 0.6663
Epoch 5/200
8000/8000 [==============================] - 10s 1ms/step - loss: 1.2222 - accuracy: 0.6664 - val_loss: 1.5636 - val_accuracy: 0.6658
Epoch 6/200
8000/8000 [==============================] - 10s 1ms/step - loss: 1.1901 - accuracy: 0.6665 - val_loss: 1.5623 - val_accuracy: 0.6658
Epoch 7/200
8000/8000 [==============================] - 10s 1ms/step - loss

8000/8000 [==============================] - 10s 1ms/step - loss: 0.0931 - accuracy: 0.9118 - val_loss: 2.2490 - val_accuracy: 0.6698
Epoch 57/200
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0929 - accuracy: 0.9112 - val_loss: 2.2506 - val_accuracy: 0.6689
Epoch 58/200
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0942 - accuracy: 0.9110 - val_loss: 2.2510 - val_accuracy: 0.6694
Epoch 59/200
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0929 - accuracy: 0.9108 - val_loss: 2.2490 - val_accuracy: 0.6694
Epoch 60/200
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0921 - accuracy: 0.9125 - val_loss: 2.2503 - val_accuracy: 0.6696
Epoch 61/200
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0908 - accuracy: 0.9127 - val_loss: 2.2513 - val_accuracy: 0.6694
Epoch 62/200
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0926 - accuracy: 0.9111 - val_loss: 2.2469 - val_a

Epoch 112/200
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0804 - accuracy: 0.9117 - val_loss: 2.2477 - val_accuracy: 0.6698
Epoch 113/200
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0802 - accuracy: 0.9135 - val_loss: 2.2457 - val_accuracy: 0.6703
Epoch 114/200
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0801 - accuracy: 0.9115 - val_loss: 2.2427 - val_accuracy: 0.6703
Epoch 115/200
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0796 - accuracy: 0.9127 - val_loss: 2.2430 - val_accuracy: 0.6701
Epoch 116/200
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0796 - accuracy: 0.9117 - val_loss: 2.2465 - val_accuracy: 0.6704
Epoch 117/200
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0795 - accuracy: 0.9138 - val_loss: 2.2444 - val_accuracy: 0.6706
Epoch 118/200
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0797 - accuracy: 0.9125 - val_

8000/8000 [==============================] - 10s 1ms/step - loss: 0.0733 - accuracy: 0.9137 - val_loss: 2.2357 - val_accuracy: 0.6701
Epoch 168/200
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0735 - accuracy: 0.9121 - val_loss: 2.2385 - val_accuracy: 0.6703
Epoch 169/200
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0736 - accuracy: 0.9134 - val_loss: 2.2331 - val_accuracy: 0.6701
Epoch 170/200
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0727 - accuracy: 0.9126 - val_loss: 2.2397 - val_accuracy: 0.6706
Epoch 171/200
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0731 - accuracy: 0.9120 - val_loss: 2.2360 - val_accuracy: 0.6699
Epoch 172/200
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0728 - accuracy: 0.9121 - val_loss: 2.2368 - val_accuracy: 0.6706
Epoch 173/200
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0729 - accuracy: 0.9123 - val_loss: 2.2379 -

In [9]:
import matplotlib.pyplot as plt
def plot_accuracy(history):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    t = f.suptitle('Transformer Performance', fontsize=12)
    f.subplots_adjust(top=0.85, wspace=0.3)

    epoch_lenght = len(history.history['accuracy'])+1
    epoch_list = list(range(1,epoch_lenght))
    
    # Plot accuracy
    ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
    ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_xticks(np.arange(0, epoch_lenght, 20))
    ax1.set_ylabel('Accuracy Value')
    ax1.set_xlabel('Epoch')
    ax1.set_title('Accuracy')
    l1 = ax1.legend(loc="best")

    # Plot loss
    ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
    ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
    ax2.set_xticks(np.arange(0, epoch_lenght, 20))
    ax2.set_ylabel('Loss Value')
    ax2.set_xlabel('Epoch')
    ax2.set_title('Loss')
    l2 = ax2.legend(loc="best")
plot_accuracy(model_history)

# predict

In [10]:
from keras_transformer import decode

decoded_predicted = decode(
    model,
    encode_input,
    start_token=target_token_dict['<START>'],
    end_token=target_token_dict['<END>'],
    pad_token=target_token_dict['<PAD>'],
)

In [11]:
start_row = 5500
for i in range(10):
    print (source_input[start_row+i], 
           # target_tokens[start_row+i],
           target_token_dict_inv[decoded_predicted[start_row+i][1:][0]])
           # target_token_dict_inv[decoded_beam[start_row+i][1:][0]])

The beautiful girl with black hair was in the park. 黒い髪の綺麗な女の子が公園にいました。
The climate of Canada is cooler than that of Japan. カナダの気候は日本よりも寒冷である。
The climate of Canada is cooler than that of Japan. カナダの気候は日本よりも寒冷である。
The committee consists of scientists and engineers. その委員会は科学者と技術者からなる。
The couple carved their initials into the oak tree. 恋人たちは自分たちの頭文字をかしの木に刻んだ。
The day will soon come when man can travel to Mars. 人類が火星に行ける日がやがて来るだろう。
The day will soon come when man can travel to Mars. 人類が火星に行ける日がやがて来るだろう。
The desire he has had for years has been fulfilled. 彼の長年の望みが実現した。
The detective shadowed the suspect for four blocks. その刑事は容疑者を4ブロックに渡って尾行した。
The doctors told Tom that Mary would never recover. 医者たちはトムに言った。メアリーに回復の見込みはないと。


# Test a translated sentence

In [12]:
def translate_word(input_sentence):
    input_index = source_input.index(input_sentence)
    input_text = source_input[input_index]
    decoded_sentence = target_token_dict_inv[decoded_predicted[input_index][1:][0]]
    print (input_text, decoded_sentence)
    
def translate_word_test(input_sentence):
    decoded_predicted = decode(
        model,
        encode_input,
        start_token=target_token_dict['<START>'],
        end_token=target_token_dict['<END>'],
        pad_token=target_token_dict['<PAD>'],
    )
    decoded_sentence = target_token_dict_inv[decoded_predicted[input_index][1:][0]]
    print (decoded_sentence)

In [13]:
translate_word("Jump!")

Jump! ジャンプして！


In [14]:
translate_word("Run.")

Run. 走って！


In [15]:
translate_word("Close your books.")

Close your books. 本を閉じなさい。


In [16]:
translate_word("Come and help me.")

Come and help me. 助けに来てください。


In [17]:
translate_word("Life is too short to worry about things like that.")

Life is too short to worry about things like that. 人生は短いのだから、そんなことを悩んでいても仕方がない。


In [18]:
translate_word("Tom said that it probably wouldn't take too much time.")

Tom said that it probably wouldn't take too much time. 多分そんなに時間はかからないだろうとトムが言った。
